In [ ]:
import networkx as nx
#import matplotlib.pyplot as plt
#import pandas as pd
import math
from queue import Queue
import time
import statistics
import json

In [ ]:
G = nx.DiGraph()
for count, line in enumerate(open("wiki-topcats-reduced.txt")):
            grlst=line.replace("\n", "").split(sep="\t")
            G.add_edge(int(grlst[0]),int(grlst[1]))

In [ ]:
def Intersection(lst1, lst2): 
    return set(lst1).intersection(lst2)

In [ ]:
categoriesdata = {}
lst2=G.nodes()
with open('wiki-topcats-categories.txt') as f:
    for i, line in enumerate(f):
        tmp = list(map(int, (line.split(';'))[1].split()))
        if len(tmp)>=3500:
            tmp=Intersection(tmp,lst2)
            if len(tmp)>=3500:
                categoriesdata[i] =list(tmp) 
f.close()

In [ ]:
with open('categoriesdata.json', 'w') as fp:
    json.dump(categoriesdata, fp)

In [ ]:
maxsub=[0,0]
for i in categoriesdata:
    if len(categoriesdata[i])>maxsub[1]:
        maxsub[0]=categoriesdata[i]
        maxsub[1]=len(categoriesdata[i])

In [ ]:
intersectiondata={}
cleancategoriesdata={}
for i in categoriesdata:
    if i!=1047:
        cleancategoriesdata[i]=list(set(categoriesdata[i])-set(categoriesdata[i]).intersection(maxsub[0]))
        intersectiondata[i]=list(set(categoriesdata[i]).intersection(maxsub[0]))

In [ ]:
with open('cleancategoriesdata.json', 'w') as fp:
    json.dump(cleancategoriesdata, fp)
with open('intersectiondata.json', 'w') as fp:
    json.dump(intersectiondata, fp)

In [ ]:
def helper(G, source, target):
    Gpred = G.pred # all predcessors
    Gsucc = G.succ # all successors
    pred = {source: None} # dictionary for predcessor
    succ = {target: None} # dictionary for successors
    forward_fringe = [source] #list
    reverse_fringe = [target] #list
    
    while forward_fringe and reverse_fringe: #while these lists are not empty
        if len(forward_fringe) <= len(reverse_fringe):
            this_level = forward_fringe
            forward_fringe = []
            for v in this_level:
                for w in Gsucc[v]:
                    if w not in pred:
                        forward_fringe.append(w)
                        pred[w] = v
                    if w in succ:
                        return pred, succ, w
        else:
            this_level = reverse_fringe
            reverse_fringe = []
            for v in this_level:
                for w in Gpred[v]:
                    if w not in succ:
                        succ[w] = v
                        reverse_fringe.append(w)
                    if w in pred:
                        return pred, succ, w

    return None
    
def shortest_path(G, source, target):
    pred, succ, w = helper(G, source, target)
    path = []
    while w is not None:
        path.append(w)
        w = pred[w]
    path.reverse()
    w = succ[path[-1]]
    while w is not None:
        path.append(w)
        w = succ[w]
    return len(path)-1

In [ ]:
blockrank={}
for i in cleancategoriesdata:
    if i!=1047:
        start = time.time()
        for j in cleancategoriesdata[i]:
            for k in intersectiondata[i]:
                try:
                    x = shortest_path(G, j, k)
                except:
                    x = -1
                if x!=-1:
                    if i in blockrank: blockrank[i].append(x)
                    else:blockrank[i]=[x]
        end = time.time()
        if i in blockrank:
            blockrank[i]=statistics.median(blockrank[i])
        print(i, end - start)
with open('blockrank.json', 'w') as fp:
    json.dump(blockrank, fp)

In [ ]:
intersectiondata